In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from sklearn.metrics import accuracy_score, confusion_matrix


### 訓練與測試資料輸入

In [19]:
path = r'C:\workspace\Project\Smart Evaluation System for Gamers Rating from Steam'
df=pd.read_csv(rf'{path}\Steam_Games_and_DLCs_Dataset\steam_DLC_20220830.csv',header=0)
df_c=pd.read_csv(rf'{path}\Steam_Games_and_DLCs_Dataset\steam_DLC_cluster_20220831.csv',header=0)

df.head()

,appid,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,categories,...,median_forever_log,header_image,screenshots,background,movies,moviesornot,owners_avg,owners_label,owners_20000,rating_scale
0,10,Counter-Strike,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],"[574941, 7]","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",...,5.313206,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,0,15000000,9,1,1
1,20,Team Fortress Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[29],"['Multi-player', 'PvP', 'Online PvP', 'Shared/...",...,3.178054,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,0,3500000,7,1,1
2,30,Day of Defeat,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Sp...",['Valve'],['Valve'],[30],"['Multi-player', 'Valve Anti-Cheat enabled']",...,2.484907,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,0,7500000,8,1,1
3,40,Deathmatch Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[31],"['Multi-player', 'PvP', 'Online PvP', 'Shared/...",...,1.945910,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,0,7500000,8,1,1
4,50,Half-Life: Opposing Force,0,NaN,NaN,"['English', 'French', 'German', 'Korean']",['Gearbox Software'],['Valve'],[32],"['Single-player', 'Multi-player', 'Valve Anti-...",...,5.525453,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,0,7500000,8,1,1


In [ ]:
df_col = df_c.columns
df_col

Index(['appid', 'name', 'required_age', 'dlc', 'fullgame',
       'supported_languages', 'developers', 'publishers', 'packages',
       'categories',
       ...
       'owners_20000', 'rating_scale', 'K_Price_2', 'K_Price_4', 'K_Cat_2',
       'K_Time_2', 'K_Platform_3', 'K_Platform_4', 'K_Genres_16',
       'K_Genres_6'],
      dtype='object', length=125)

In [ ]:
df_corr = df_c.corr(method='pearson')
df_corr.columns

Index(['appid', 'required_age', 'achievements', 'coming_soon', 'price',
       'review_score', 'total_positive', 'total_negative', 'rating',
       'average_forever',
       ...
       'owners_20000', 'rating_scale', 'K_Price_2', 'K_Price_4', 'K_Cat_2',
       'K_Time_2', 'K_Platform_3', 'K_Platform_4', 'K_Genres_16',
       'K_Genres_6'],
      dtype='object', length=105)

In [7]:
labels=df_c['rating_scale']

In [4]:
feature_cols = ['required_age', 'achievements', 'coming_soon', 'price',
       'average_forever', 'median_forever', 'type_advertising', 'type_dlc',
       'type_game', 'type_music', 'is_required_age', 'required_age_log',
       'supported_languages_count', 'supported_languages_count_log',
       'developer_label', 'publisher_label', 'developer_count',
       'developer_size', 'publisher_count', 'publisher_size',
       'developers_publishers_same', 'developer_count_log',
       'publisher_count_log', 'platforms_linux_only', 'platforms_mac_linux',
       'platforms_mac_only', 'platforms_windows_linux',
       'platforms_windows_mac_linux', 'platforms_windows_mac',
       'platforms_windows_only', 'platforms_label', 'platforms_windows',
       'platforms_mac', 'platforms_linux', 'categories_Multi_player',
       'categories_Single_player', 'categories_Anti_Cheat_enabled',
       'categories_Co-op', 'categories_PvP', 'genres_Strategy',
       'genres_Sports', 'genres_Episodic', 'genres_Nudity', 'genres_Movie',
       'genres_Utilities', 'genres_Casual', 'genres_Indie',
       'genres_Software Training', 'genres_Action',
       'genres_Massively Multiplayer', 'genres_Education',
       'genres_Early Access', 'genres_Web Publishing', 'genres_Free to Play',
       'genres_Design & Illustration', 'genres_Others', 'genres_Photo Editing',
       'genres_Accounting', 'genres_Game Development', 'genres_RPG',
       'genres_Sexual Content', 'genres_Animation & Modeling', 'genres_Racing',
       'genres_360 Video', 'genres_Simulation', 'genres_Tutorial',
       'genres_Documentary', 'genres_Video Production', 'genres_Adventure',
       'genres_Violent', 'genres_Audio Production', 'genres_Gore',
       'genres_Short', 'achievements_log', 'is_achievements', 'leadtime',
       'leadtime_log', 'release_month', 'release_year', 'release_quarter',
       'supported_audio_count', 'is_supported_audio', 'is_coming_soon',
       'price_log', 'is_free', 'average_forever_log', 'median_forever_log',
       'moviesornot', 'owners_avg',
       'owners_label', 'owners_20000', 'K_Price_2', 'K_Price_4', 'K_Cat_2',
       'K_Time_2', 'K_Platform_3', 'K_Platform_4', 'K_Genres_16',
       'K_Genres_6']

In [5]:
data = df_c[feature_cols]

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels,test_size=0.33, random_state=101)

In [9]:
data.columns

Index(['required_age', 'achievements', 'coming_soon', 'price',
       'average_forever', 'median_forever', 'type_advertising', 'type_dlc',
       'type_game', 'type_music', 'is_required_age', 'required_age_log',
       'supported_languages_count', 'supported_languages_count_log',
       'developer_label', 'publisher_label', 'developer_count',
       'developer_size', 'publisher_count', 'publisher_size',
       'developers_publishers_same', 'developer_count_log',
       'publisher_count_log', 'platforms_linux_only', 'platforms_mac_linux',
       'platforms_mac_only', 'platforms_windows_linux',
       'platforms_windows_mac_linux', 'platforms_windows_mac',
       'platforms_windows_only', 'platforms_label', 'platforms_windows',
       'platforms_mac', 'platforms_linux', 'categories_Multi_player',
       'categories_Single_player', 'categories_Anti_Cheat_enabled',
       'categories_Co-op', 'categories_PvP', 'genres_Strategy',
       'genres_Sports', 'genres_Episodic', 'genres_Nudity', 'g

In [ ]:
X_train,y_train

(       english  required_age  achievements  average_playtime  median_playtime  \
 22244        1             0            16                 0                0   
 3358         1             0            57                 4                4   
 24368        1             0             0                 0                0   
 14428        1             0           153                 0                0   
 2333         1             0             1                 0                0   
 ...        ...           ...           ...               ...              ...   
 5695         1             0             0               219              224   
 8006         1             0             0                 0                0   
 17745        1             0             0                 0                0   
 17931        1             0            63                 0                0   
 13151        1             0             0                 0                0   
 
        price 

In [ ]:
X_test,y_test

(       english  required_age  achievements  average_playtime  median_playtime  \
 20465        1             0             0                 0                0   
 12671        1             0            25                 0                0   
 20716        1             0             0                 0                0   
 6984         1             0            10                84              139   
 11174        1             0             8                 0                0   
 ...        ...           ...           ...               ...              ...   
 20416        1             0            10                 0                0   
 5608         1             0            10               203              203   
 25           1             0           167             22494             6502   
 12457        1             0             0                 0                0   
 11763        1             0            50               266              266   
 
        price 

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107360 entries, 0 to 107359
Data columns (total 99 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       107360 non-null  object
 1   1       107360 non-null  object
 2   2       107360 non-null  object
 3   3       107360 non-null  object
 4   4       107360 non-null  object
 5   5       107360 non-null  object
 6   6       107360 non-null  object
 7   7       107360 non-null  object
 8   8       107360 non-null  object
 9   9       107360 non-null  object
 10  10      107360 non-null  object
 11  11      107360 non-null  object
 12  12      107360 non-null  object
 13  13      107360 non-null  object
 14  14      107360 non-null  object
 15  15      107360 non-null  object
 16  16      107360 non-null  object
 17  17      107360 non-null  object
 18  18      107360 non-null  object
 19  19      107360 non-null  object
 20  20      107360 non-null  object
 21  21      107360 non-null  object
 

# **訓練集類別平衡**

In [ ]:
train_data = X_train
train_data['rating_scale'] = pd.Series(y_train)

In [ ]:
train_data_y = train_data['rating_scale']
train_data_y

102249    0
89281     0
6993      0
46652     0
88295     0
         ..
5695      0
73542     1
83281     0
83467     0
45919     0
Name: rating_scale, Length: 68677, dtype: int64

In [ ]:
print('0:', np.sum( train_data_y == 0 ))
print('1:', np.sum( train_data_y == 1 ))

0: 53680
1: 14997


In [ ]:
max(np.sum( train_data_y == 0 ), np.sum( train_data_y == 1 ))

53680

In [ ]:
train_data_y_0 = train_data[train_data['rating_scale'] == 0]
train_data_y_1 = train_data[train_data['rating_scale'] == 1]

少的一邊用隨機抽後放回補至跟多的一樣

In [ ]:
train_data_y_1_re = train_data_y_1.sample(max(np.sum( train_data_y == 0 ), np.sum( train_data_y == 1 )), replace = True)

In [ ]:
train_data_new = pd.concat([train_data_y_0, train_data_y_1_re], axis=0)

In [ ]:
X_train_new = train_data_new.drop('rating_scale', axis = 1)
y_train_new = train_data_new['rating_scale']

In [ ]:
y_train_new

102249    0
89281     0
6993      0
46652     0
88295     0
         ..
1530      1
92317     1
73713     1
30101     1
36233     1
Name: rating_scale, Length: 107360, dtype: int64

In [ ]:
X_train = pd.DataFrame(X_train_new)

In [ ]:
y_train = pd.Series(y_train_new)

# CART

In [10]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [11]:
# Create Decision Tree classifer object
Cart = DecisionTreeClassifier(criterion="gini", #CART-->gini c4.5-->entropy
                            max_depth=50, #max_depth=4
                            min_samples_leaf=600, #min_samples_leaf=20
                            #class_weight='balanced' # 'balanced', None
                            )   

# Train Decision Tree Classifer
Cart.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=50, min_samples_leaf=600)

In [12]:
Cart_train_score=Cart.score(X_train, y_train)
print('The Accuracy of CART Classifier on training set:', Cart_train_score)

The Accuracy of CART Classifier on training set: 0.8176536540617674


In [ ]:
from sklearn.metrics import classification_report

Cart_pred_train =Cart.predict(X_train)

print(classification_report(y_train,Cart_pred_train))
print(confusion_matrix(y_train,Cart_pred_train))

              precision    recall  f1-score   support

           0       0.78      0.71      0.74     53680
           1       0.73      0.80      0.76     53680

    accuracy                           0.75    107360
   macro avg       0.75      0.75      0.75    107360
weighted avg       0.75      0.75      0.75    107360

[[37882 15798]
 [10901 42779]]


In [ ]:
Cart_test_score=Cart.score(X_test,y_test)
print('The Accuracy of CART/C4.5 Classifier on testing set:', Cart_test_score)

The Accuracy of CART/C4.5 Classifier on testing set: 0.7166169036568422


In [ ]:
from sklearn.metrics import classification_report

Cart_pred =Cart.predict(X_test)

print(classification_report(y_test,Cart_pred))
print(confusion_matrix(y_test,Cart_pred))

              precision    recall  f1-score   support

           0       0.92      0.70      0.79     26403
           1       0.42      0.78      0.55      7424

    accuracy                           0.72     33827
   macro avg       0.67      0.74      0.67     33827
weighted avg       0.81      0.72      0.74     33827

[[18437  7966]
 [ 1620  5804]]


# C45

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [ ]:
# Create Decision Tree classifer object
#CartC45 = DecisionTreeClassifier(criterion="gini", max_depth=50, min_samples_leaf=10)
C45 = DecisionTreeClassifier(criterion="entropy", max_depth=50, min_samples_leaf=600, class_weight=None)  #CART-->gini c4.5-->entropy min_samples_leaf=20, max_depth=4, class_weight='balanced',None

# Train Decision Tree Classifer
C45.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=50, min_samples_leaf=600)

In [ ]:
C45_train_score=Cart.score(X_train, y_train)
print('The Accuracy of C4.5 Classifier on training set:', C45_train_score)

The Accuracy of C4.5 Classifier on training set: 0.8176536540617674


In [ ]:
from sklearn.metrics import classification_report

C45_pred_train =C45.predict(X_train)

print(classification_report(y_train,C45_pred_train))
print(confusion_matrix(y_train,C45_pred_train))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89     53680
           1       0.65      0.34      0.45     14997

    accuracy                           0.82     68677
   macro avg       0.74      0.65      0.67     68677
weighted avg       0.80      0.82      0.79     68677

[[50876  2804]
 [ 9824  5173]]


In [ ]:
C45test_score=C45.score(X_test,y_test)
print('The Accuracy of C4.5 Classifier on testing set:', C45test_score)

The Accuracy of C4.5 Classifier on testing set: 0.8113637035504183


In [ ]:
from sklearn.metrics import classification_report

C45_pred =C45.predict(X_test)

print(classification_report(y_test,C45_pred))
print(confusion_matrix(y_test,C45_pred))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89     26403
           1       0.63      0.33      0.44      7424

    accuracy                           0.81     33827
   macro avg       0.73      0.64      0.66     33827
weighted avg       0.79      0.81      0.79     33827

[[24979  1424]
 [ 4957  2467]]


# ExtraTrees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ETreec = ExtraTreesClassifier(max_depth=20, # 樹的深度
                            n_estimators=1500, #k值_決策樹數量
                            criterion='gini', # gini, entropy
                            bootstrap=True, # 樹的製作抽後放回
                            min_samples_split=800, #每一節點最小樣本數 
                            random_state=0, # 亂樹種子
                            class_weight=None) #None, balanced, balanced_subsample

In [ ]:
ETreec.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, max_depth=20, min_samples_split=800,
                     n_estimators=1500, random_state=0)

In [ ]:
ETreetrain_score=ETreec.score(X_train, y_train)
print('The Accuracy of ExtraTrees Classifier on training set:', ETreetrain_score)

The Accuracy of ExtraTrees Classifier on training set: 0.8191534283675758


In [ ]:
ETree_pred=ETreec.predict(X_train)
print(classification_report(y_train,ETree_pred))
print(confusion_matrix(y_train,ETree_pred))

              precision    recall  f1-score   support

           0       0.83      0.97      0.89     53680
           1       0.72      0.28      0.40     14997

    accuracy                           0.82     68677
   macro avg       0.78      0.62      0.65     68677
weighted avg       0.81      0.82      0.79     68677

[[52089  1591]
 [10829  4168]]


In [ ]:
ETreetest_score=ETreec.score(X_test,y_test)
print('The Accuracy of ExtraTrees Classifier on testing set:', ETreetest_score)


The Accuracy of ExtraTrees Classifier on testing set: 0.8181630058828746


In [ ]:
ETree_pred=ETreec.predict(X_test)
print(classification_report(y_test,ETree_pred))
print(confusion_matrix(y_test,ETree_pred))

              precision    recall  f1-score   support

           0       0.83      0.97      0.89     26403
           1       0.73      0.27      0.40      7424

    accuracy                           0.82     33827
   macro avg       0.78      0.62      0.64     33827
weighted avg       0.80      0.82      0.78     33827

[[25658   745]
 [ 5406  2018]]


# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=1000,
                     criterion='gini',
                     max_depth=None,
                     bootstrap=True, ccp_alpha=0.0, class_weight=None, # 'balanced', None
                     max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0,
                     min_samples_leaf=50, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, 
                     n_jobs=None, oob_score=False, random_state=None,
                     verbose=0, warm_start=False, )


In [ ]:
rfc.fit(X_train, y_train)

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_features='auto', min_samples_leaf=50,
                       n_estimators=1000)

In [ ]:
RFtrain_score=rfc.score(X_train, y_train)
print('The Accuracy of RandomForest Classifier on training set:', RFtrain_score)

The Accuracy of RandomForest Classifier on training set: 0.8321272041585975


In [ ]:
RF_pred=rfc.predict(X_train)
print(classification_report(y_train,RF_pred))
print(confusion_matrix(y_train,RF_pred))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90     53680
           1       0.76      0.34      0.47     14997

    accuracy                           0.83     68677
   macro avg       0.80      0.65      0.68     68677
weighted avg       0.82      0.83      0.81     68677

[[52084  1596]
 [ 9933  5064]]


In [ ]:
RFtest_score=rfc.score(X_test,y_test)
print('The Accuracy of RandomForest Classifier on testing set:', RFtest_score)

The Accuracy of RandomForest Classifier on testing set: 0.8233955124604606


In [ ]:
RF_pred=rfc.predict(X_test)
print(classification_report(y_test,RF_pred))
print(confusion_matrix(y_test,RF_pred))

              precision    recall  f1-score   support

           0       0.83      0.97      0.90     26403
           1       0.73      0.31      0.44      7424

    accuracy                           0.82     33827
   macro avg       0.78      0.64      0.67     33827
weighted avg       0.81      0.82      0.79     33827

[[25531   872]
 [ 5102  2322]]


# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
AdaBoostc = AdaBoostClassifier(random_state=0,n_estimators=500, learning_rate=1,)

In [ ]:
AdaBoostc.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
AdaBoosttrain_score=AdaBoostc.score(X_train, y_train)
print('The Accuracy of AdaBoost Classifier on testing set:', AdaBoosttrain_score)

The Accuracy of AdaBoost Classifier on testing set: 0.6843527659809167


In [ ]:
AdaBoostc_pred=AdaBoostc.predict(X_train)
print(classification_report(y_train,AdaBoostc_pred))
print(confusion_matrix(y_train,AdaBoostc_pred))

              precision    recall  f1-score   support

           0       0.70      0.87      0.77     11299
           1       0.64      0.38      0.48      6832

    accuracy                           0.68     18131
   macro avg       0.67      0.62      0.63     18131
weighted avg       0.67      0.68      0.66     18131

[[9805 1494]
 [4229 2603]]


In [ ]:
AdaBoosttest_score=AdaBoostc.score(X_test,y_test)
print('The Accuracy of AdaBoost Classifier on testing set:', AdaBoosttest_score)

The Accuracy of AdaBoost Classifier on testing set: 0.6761840779308028


In [ ]:
AdaBoostc_pred=AdaBoostc.predict(X_test)
print(classification_report(y_test,AdaBoostc_pred))
print(confusion_matrix(y_test,AdaBoostc_pred))

              precision    recall  f1-score   support

           0       0.69      0.85      0.76      5491
           1       0.63      0.39      0.48      3440

    accuracy                           0.68      8931
   macro avg       0.66      0.62      0.62      8931
weighted avg       0.67      0.68      0.66      8931

[[4693  798]
 [2094 1346]]


# **XGBOOST**

In [ ]:
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier(        
        n_estimators=100,     #樹的個數
        learning_rate= 0.3,    # 如同學習率 
        max_depth=50,       # 構建樹的深度，越大越容易過擬合    
        subsample=1,         # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0.2,         # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=1,          # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0,        #最大增量步長，我們允許每個樹的權重估計。
        colsample_bytree=1,         # 生成樹時進行的列取樣 
        min_child_weight=1, 
                     # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
                     # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
                     #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        seed=1000         #隨機種子
#        reg_alpha=0,        # L1 正則項引數
        #scale_pos_weight=1,        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #objective= 'multi:softmax',         #多分類的問題 指定學習任務和相應的學習目標
        #num_class=10,        # 類別數，多分類與 multisoftmax 並用
#        silent=0 ,        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        nthread=4,        # cpu 執行緒數 預設最大
        #eval_metric= 'auc'
)


In [ ]:
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=50, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=1000, reg_alpha=0, reg_lambda=1, ...)

In [ ]:
XGBtrain_score=xgbc.score(X_train, y_train)
print('The Accuracy of XGBOOST Classifier on testing set:', XGBtrain_score)

The Accuracy of XGBOOST Classifier on testing set: 0.8092217748607358


In [ ]:
XGB_pred=xgbc.predict(X_train)
print(classification_report(y_train,XGB_pred))
print(confusion_matrix(y_train,XGB_pred))

              precision    recall  f1-score   support

           0       0.79      0.95      0.86     11299
           1       0.87      0.58      0.70      6832

    accuracy                           0.81     18131
   macro avg       0.83      0.76      0.78     18131
weighted avg       0.82      0.81      0.80     18131

[[10680   619]
 [ 2840  3992]]


In [ ]:
XGBtest_score=xgbc.score(X_test,y_test)
print('The Accuracy of XGBOOST Classifier on testing set:', XGBtest_score)

The Accuracy of XGBOOST Classifier on testing set: 0.6602844026424812


In [ ]:
XGB_pred=xgbc.predict(X_test)
print(classification_report(y_test,XGB_pred))
print(confusion_matrix(y_test,XGB_pred))

              precision    recall  f1-score   support

           0       0.69      0.81      0.75      5491
           1       0.58      0.42      0.49      3440

    accuracy                           0.66      8931
   macro avg       0.64      0.62      0.62      8931
weighted avg       0.65      0.66      0.65      8931

[[4455 1036]
 [1998 1442]]


# **LightGBM**

## **Python API**

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelBinarizer


In [ ]:
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier()


In [ ]:
lgbmc.fit(X_train, y_train)

LGBMClassifier()

In [ ]:
LGBMtrain_score=lgbmc.score(X_train, y_train)
print('The Accuracy of LightGBM Classifier on training set:', LGBMtrain_score)

The Accuracy of LightGBM Classifier on training set: 0.7200926589818543


In [ ]:
LGBM_pred=lgbmc.predict(X_train)
print(classification_report(y_train,LGBM_pred))
print(confusion_matrix(y_train,LGBM_pred))

              precision    recall  f1-score   support

           0       0.72      0.89      0.80     11299
           1       0.71      0.44      0.54      6832

    accuracy                           0.72     18131
   macro avg       0.72      0.66      0.67     18131
weighted avg       0.72      0.72      0.70     18131

[[10077  1222]
 [ 3853  2979]]


In [ ]:
LGBMtest_score=lgbmc.score(X_test,y_test)
print('The Accuracy of LightGBM Classifier on testing set:', LGBMtest_score)

The Accuracy of LightGBM Classifier on testing set: 0.6762960474750868


In [ ]:
LGBM_pred=lgbmc.predict(X_test)
print(classification_report(y_test,LGBM_pred))
print(confusion_matrix(y_test,LGBM_pred))

              precision    recall  f1-score   support

           0       0.69      0.85      0.76      5491
           1       0.63      0.40      0.49      3440

    accuracy                           0.68      8931
   macro avg       0.66      0.62      0.63      8931
weighted avg       0.67      0.68      0.66      8931

[[4668  823]
 [2068 1372]]


## **scikit-learn**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GradientBoost = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=10, random_state=0, )

In [ ]:
GradientBoost.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=10, random_state=0)

In [ ]:
GradientBoostMtrain_score=GradientBoost.score(X_train, y_train)
print('The Accuracy of GradientBoost Classifier on training set:', GradientBoostMtrain_score)

The Accuracy of LightGBM Classifier on training set: 0.8162815068115382


In [ ]:
GradientBoost_pred=GradientBoost.predict(X_train)
print(classification_report(y_train,GradientBoost_pred))
print(confusion_matrix(y_train,GradientBoost_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.87     11299
           1       0.87      0.60      0.71      6832

    accuracy                           0.82     18131
   macro avg       0.83      0.77      0.79     18131
weighted avg       0.82      0.82      0.81     18131

[[10673   626]
 [ 2705  4127]]


In [ ]:
GradientBoosttest_score=GradientBoost.score(X_test,y_test)
print('The Accuracy of GradientBoost Classifier on testing set:', GradientBoosttest_score)

The Accuracy of LightGBM Classifier on testing set: 0.6494233568469376


In [ ]:
GradientBoost_pred=GradientBoost.predict(X_test)
print(classification_report(y_test,GradientBoost_pred))
print(confusion_matrix(y_test,GradientBoost_pred))

              precision    recall  f1-score   support

           0       0.68      0.80      0.74      5491
           1       0.56      0.41      0.48      3440

    accuracy                           0.65      8931
   macro avg       0.62      0.61      0.61      8931
weighted avg       0.64      0.65      0.64      8931

[[4379 1112]
 [2019 1421]]


# ***CatBOOST***



In [ ]:
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('ggplot') 
import seaborn as sns
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold


In [ ]:
CatBc = CatBoostClassifier(iterations=20,
  learning_rate=1,
  depth=10,
  loss_function='MultiClass', 
  auto_class_weights=None)

In [ ]:
CatBc.fit(X_train, y_train)

0:	learn: 0.6066531	total: 196ms	remaining: 3.72s
1:	learn: 0.5956649	total: 246ms	remaining: 2.21s
2:	learn: 0.5927437	total: 296ms	remaining: 1.68s
3:	learn: 0.5879724	total: 346ms	remaining: 1.38s
4:	learn: 0.5877482	total: 352ms	remaining: 1.06s
5:	learn: 0.5859823	total: 403ms	remaining: 939ms
6:	learn: 0.5856433	total: 408ms	remaining: 757ms
7:	learn: 0.5844361	total: 463ms	remaining: 694ms
8:	learn: 0.5832356	total: 513ms	remaining: 627ms
9:	learn: 0.5820626	total: 565ms	remaining: 565ms
10:	learn: 0.5801547	total: 619ms	remaining: 506ms
11:	learn: 0.5787891	total: 671ms	remaining: 447ms
12:	learn: 0.5768658	total: 720ms	remaining: 388ms
13:	learn: 0.5753374	total: 772ms	remaining: 331ms
14:	learn: 0.5727568	total: 816ms	remaining: 272ms
15:	learn: 0.5715333	total: 860ms	remaining: 215ms
16:	learn: 0.5695713	total: 902ms	remaining: 159ms
17:	learn: 0.5680220	total: 944ms	remaining: 105ms
18:	learn: 0.5669732	total: 986ms	remaining: 51.9ms
19:	learn: 0.5656726	total: 1.04s	remain

In [ ]:
CatBtrain_score=CatBc.score(X_train, y_train)
print('The Accuracy of CatBOOST Classifier on training set:',CatBtrain_score)

The Accuracy of CatBOOST Classifier on training set: 0.7109922232640229


In [ ]:
CatB_pred=CatBc.predict(X_train)
print(classification_report(y_train,CatB_pred))
print(confusion_matrix(y_train,CatB_pred))

              precision    recall  f1-score   support

           0       0.72      0.88      0.79     11299
           1       0.68      0.43      0.53      6832

    accuracy                           0.71     18131
   macro avg       0.70      0.66      0.66     18131
weighted avg       0.71      0.71      0.69     18131

[[9940 1359]
 [3881 2951]]


In [ ]:
CatBest_score=CatBc.score(X_test,y_test)
print('The Accuracy of CatBOOST Classifier on testing set:',CatBest_score)

The Accuracy of CatBOOST Classifier on testing set: 0.6715933266151607


In [ ]:
CatB_pred=CatBc.predict(X_test)
print(classification_report(y_test,CatB_pred))
print(confusion_matrix(y_test,CatB_pred))

              precision    recall  f1-score   support

           0       0.69      0.84      0.76      5491
           1       0.61      0.40      0.49      3440

    accuracy                           0.67      8931
   macro avg       0.65      0.62      0.62      8931
weighted avg       0.66      0.67      0.65      8931

[[4614  877]
 [2056 1384]]


# **voting system**

In [ ]:
y1 = Cart_pred
y2 = C45_pred
y3 = ETree_pred
y4 = RF_pred
y5 = AdaBoostc_pred
y6 = XGB_pred
y7 = LGBM_pred
y8 = GradientBoost_pred
y9 = CatB_pred

In [ ]:
y_final = y1+y2+y3+y4+y5+y6+y7+y8+y9

In [ ]:
print(classification_report(y_test,y_final))
print(confusion_matrix(y_test,y_final))

# **model storage**

In [15]:
from joblib import dump

In [20]:
dump(Cart, fr'{path}\model\Cart.joblib')

['C:\\workspace\\Project\\Smart Evaluation System for Gamers Rating from Steam\\model\\Cart.joblib']